In [1]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp
import pulp as plp


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def optimise_gw38_fpl_challenge(fplreview, bench_multiplier, budget=1000):
    """
    Use PuLP to optimise the FPL GW38 challenge. The FPL GW38 challenge is a maximisation problem,
    with normal FPL rules, but the 15 man squad can only contain a single player from each team.

    Args:
        fplreview (DataFrame): The FPL review data
        bench_multiplier (float): The multiplier for the bench size
        budget (int): The budget for the team

    Returns:
        Tuple[List[LpVariable], List[LpVariable], List[LpVariable]]: The decision variables for lineup, bench, and captaincy
    """

    # Get the number of players and their list of ids
    player_ids = fplreview['ID'].tolist()
    player_count = len(player_ids)

    # Set up the problem
    model = plp.LpProblem("fpl-gw38-challenge", plp.LpMaximize)

    # Define the decision variables
    # We need to decide on a lineup, a bench and a captaincy choice
    lineup = [
        plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
        for i in player_ids
    ]
    bench = [
        plp.LpVariable(f"bench_{i}", lowBound=0, upBound=1, cat="Integer")
        for i in player_ids
    ]
    captaincy = [
        plp.LpVariable(f"captaincy_{i}", lowBound=0, upBound=1, cat="Integer")
        for i in player_ids
    ]

    # Set the objective function (the number of points scored by the team)
    model += sum((lineup[i] + captaincy[i] + + (bench_multiplier * bench[i])) * fplreview["38_Pts"][i] for i in range(player_count))

    # Set the constraints (the team must be within budget)
    model += sum((lineup[i] + bench[i]) * fplreview["BV"][i] for i in range(player_count)) <= budget

    # G constraints (there must be 1 goalkeeper in the 11 man lineup, and 2 in the 15 man squad)
    model += sum(lineup[i] for i in range(player_count) if fplreview['Pos'][i] == 'G') == 1
    model += sum(lineup[i] + bench[i] for i in range(player_count) if fplreview['Pos'][i] == 'G') == 2

    # D constraints (there must be at least 3 defenders in the 11 man lineup, and exactly 5 in the 15 man squad)
    model += sum(lineup[i] for i in range(player_count) if fplreview['Pos'][i] == 'D') >= 3
    model += sum(lineup[i] + bench[i] for i in range(player_count) if fplreview['Pos'][i] == 'D') == 5

    # M constraints (there must be at least 2 midfielders in the 11 man lineup, and exactly 5 in the 15 man squad)
    model += sum(lineup[i] for i in range(player_count) if fplreview['Pos'][i] == 'M') >= 2
    model += sum(lineup[i] + bench[i] for i in range(player_count) if fplreview['Pos'][i] == 'M') == 5

    # F constraints (there must be at least 1 forward in the 11 man lineup, and exactly 3 in the 15 man squad)
    model += sum(lineup[i] for i in range(player_count) if fplreview['Pos'][i] == 'F') >= 1
    model += sum(lineup[i] + bench[i] for i in range(player_count) if fplreview['Pos'][i] == 'F') == 3

    # Team constraints - There must be 11 in the lineup, and 4 on the bench, with a single captain.
    model += sum(lineup) == 11
    model += sum(bench) == 4
    model += sum(lineup) + sum(bench) == 15
    model += sum(captaincy) == 1
    for teams in fplreview['Team'].unique():
        # The key stipulation for this gameweek's challenge, only a single player from each team.
        model += sum(lineup[i] + bench[i] for i in range(player_count) if fplreview['Team'][i] == teams) <= 1
    
    # A player cannot start and be benched, and the player who is the captain can't be on the bench.
    for i in range(player_count):
        model += (lineup[i] + bench[i]) <= 1
        model += (lineup[i] - captaincy[i]) >= 0

    # Solve the problem
    plp.LpSolverDefault.msg = 0
    model.solve()

    # Return the values
    return lineup, bench, captaincy, 

In [3]:
def print_result(lineup, bench, captaincy, fplreview):
    """
    Prints the results of the optimisation.

    Args:
        lineup (List[LpVariable]): The decision variables for the lineup.
        bench (List[LpVariable]): The decision variables for the bench.
        captaincy (List[LpVariable]): The decision variables for the captaincy.
    """
    print('------------------')
    print('Starting Lineup:')
    print('------------------')

    # Running totals for optimal xPts and the budget spent
    total_xpts = 0
    total_budget = 0
    for i in range(len(lineup)):
        # If the player is in the lineup (value of 1 (True), then add their points and budget totals)
        if lineup[i].varValue == 1:
            print(f"{fplreview['Name'][i]}: {fplreview['Pos'][i]}")
            total_xpts += fplreview['38_Pts'][i]
            total_budget += fplreview['BV'][i]

    print('------------------')
    print('Bench:')
    print('------------------')
    for i in range(len(bench)):
        # If the player is on the bench, they still need to be budgeted for, but their points
        # will not be considered
        if bench[i].varValue == 1:
            print(f"{fplreview['Name'][i]}: {fplreview['Pos'][i]}")
            total_budget += fplreview['BV'][i]

    print('------------------')
    print('Captaincy:')
    print('------------------')
    for i in range(len(captaincy)):
        # Captaincy values are considered in the lineup (as we know our constraints stipulate that the
        # captain must be in the lineup. To account for their double points, we just need to re-add their points
        # once again. Their budget is already accounted for.)
        if captaincy[i].varValue == 1:
            print(f"{fplreview['Name'][i]}: {fplreview['Pos'][i]}")
            total_xpts += fplreview['38_Pts'][i]

    # Prints the final values.
    print('Total xPts:', total_xpts )
    print('Total Budget Spent:', total_budget)

In [4]:
# Reading in the data for GW38 (downloaded from fplreview.com)
fplreview = pd.read_csv('fplreview_gw38_202324.csv')
fplreview = fplreview.drop(['SV', 'Elite%'], axis=1)
display(fplreview)

# Runs the optimisation (all games starting at the same time, with early team news, means we can use a bench
# multiplier of 0, and just manually adjust xPts values if we discover a player is not playing.) 
lineup, bench, captaincy = optimise_gw38_fpl_challenge(fplreview, bench_multiplier=0.0, budget=100)

# Print the result.
print_result(lineup, bench, captaincy, fplreview)

,Pos,ID,Name,BV,Team,38_xMins,38_Pts
0,F,1,Balogun,4.4,Arsenal,0,0.00
1,D,2,Cédric,3.8,Arsenal,0,0.00
2,M,3,M.Elneny,4.4,Arsenal,1,0.05
3,M,4,Fábio Vieira,5.4,Arsenal,7,0.53
4,D,5,Gabriel,5.4,Arsenal,92,4.44
...,...,...,...,...,...,...,...
862,D,10142,Rhys Williams,99.9,Liverpool,0,0.00
863,M,10147,Yunus Emre Konak,99.9,Brentford,0,0.00
864,G,10153,Hákon Rafn Valdimarsson,99.9,Brentford,0,0.00
865,F,10163,Rodrigo Ribeiro,99.9,Nott'm Forest,0,0.00


------------------
Starting Lineup:
------------------
Gabriel: D
Toney: F
Eze: M
Leno: G
Salah: M
Doughty: D
Haaland: F
Palmer: M
B.Fernandes: M
Son: M
Assignon: D
------------------
Bench:
------------------
Turner: G
Chambers: D
Lascelles: D
Mubama: F
------------------
Captaincy:
------------------
Haaland: F
Total xPts: 60.769999999999996
Total Budget Spent: 100.0
